In [1]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd

In [2]:
#set up executable path & the url for scraping
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/kimberlybraud/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [3]:
#assign the url and instruct the browser to visit it.
url = 'https://redplanetscience.com'
browser.visit(url)


In [4]:
# Optional delay of 1 second before searching for components
#The optional delay is useful because sometimes dynamic pages take a little while to load,
#especially if they are image-heavy.
browser.is_element_present_by_css('div.list_text', wait_time=1)

True

In [5]:
# set up HTML parser

html = browser.html
news_soup = soup(html, 'html.parser')
slide_elem = news_soup.select_one('div.list_text')

#slide_elem as the variable to look for the <div /> tag and its descendent (the other tags within the <div /> element). 
#This is our parent element
#The . is used for selecting classes, such as list_text, so the code 'div.list_text' pinpoints the <div /> tag with the class of list_text

In [6]:
#assign the title and summary text to variables we'll reference later
slide_elem.find('div', class_='content_title')

<div class="content_title">A New Video Captures the Science of NASA's Perseverance Mars Rover</div>

In [7]:
#get just the text - not the extra HTML stuff
news_title = slide_elem.find('div', class_='content_title').get_text()
news_title

"A New Video Captures the Science of NASA's Perseverance Mars Rover"

In [8]:
#get just the article teaser text
news_p = slide_elem.find('div', class_= 'article_teaser_body').get_text()
news_p

'With a targeted launch date of July 30, the next robotic scientist NASA is sending to the to the Red Planet has big ambitions.'

### Featured Images

In [9]:
# Visit URL
url = 'https://spaceimages-mars.com'
browser.visit(url)

In [10]:
# Find and click the full image button
full_image_elem = browser.find_by_tag('button')[1]
full_image_elem.click()

In [11]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [12]:
#use the image tag and class (<img />and fancybox-img) 
#to build the URL to the full-size image.

# Find the relative image url
img_url_rel = img_soup.find('img', class_='fancybox-image').get('src')
img_url_rel

'image/featured/mars1.jpg'

In [13]:
# add the base URL to our code. / Use the base URL to create an absolute URL
img_url = f'https://spaceimages-mars.com/{img_url_rel}'
img_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [14]:
#scrape the entire table with pandas' .read_html()
df = pd.read_html('https://galaxyfacts-mars.com')[0]
df.columns = ['description', 'Mars', 'Earth']
df.set_index('description', inplace = True)
df

,Mars,Earth
description,,
Mars - Earth Comparison,Mars,Earth
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [15]:
# convert df back to html-ready code
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <td>Mars</td>\n      <td>Earth</td>\n    </tr>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:</th>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>Temperature:</th>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>

## Practice

In [16]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
from re import findall

from urllib.parse import urljoin

In [17]:
link = 'https://marshemispheres.com/'

In [18]:
#grab data from site
pg_response = requests.get(link, timeout = 5)

In [19]:
#structure raw data so I can parse it
pg_content = BeautifulSoup(pg_response.content)

In [20]:
#loop through each tag and grab its content
titles_list = []
image_urls_list = []

a_tags = pg_content.find_all('a', class_ = 'itemLink')
a_tags

[<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="cerberus.html">
 <h3>Cerberus Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="schiaparelli.html">
 <h3>Schiaparelli Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="syrtis.html"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="syrtis.html">
 <h3>Syrtis Major Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="valles.html"><

In [21]:
base_url = 'https://marshemispheres.com/'
titles_list = []
image_urls_list = []
img_url = []

for i in a_tags:
    titles = i.find('h3').text
    links = i.find('img', class_ = 'thumb')
  
    if titles != []:
        titles_list.append(titles)
        image_urls_list.append(links.get('src'))
        
        # for i in image_urls_list:
        #     if img_url != []:
        #         urls = urljoin(base_url, {image_urls_list})
        #         img_url.append(urls)

        # for i in image_urls_list:
        #     if img_url != []:
        #         urls = f'https://astrogeology.usgs.gov/{image_urls_list}'
        #         img_url.append(urls)
        
image_urls_list

AttributeError: 'NoneType' object has no attribute 'text'

In [22]:
final_titles_list = titles_list[0:4]
final_titles_list

[]

In [23]:
hemispheres = dict(zip(final_titles_list, image_urls_list))
hemispheres

{}

## Practice Round 2

In [38]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
from webdriver_manager.chrome import ChromeDriverManager
from re import findall

In [39]:
# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 92.0.4515
Get LATEST driver version for 92.0.4515
Driver [/Users/kimberlybraud/.wdm/drivers/chromedriver/mac64/92.0.4515.107/chromedriver] found in cache


In [40]:
# 1. Use browser to visit the URL# Visit the site
url = 'https://marshemispheres.com/'
browser.visit(url)

In [41]:
# Parse the HTML
html = browser.html
html_soup = soup(html, 'html.parser')
html_soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [42]:
#2. Create a list to hold the images and titles
hemisphere_image_urls = []

a_tags = html_soup.find_all('a', class_ = 'itemLink')
a_tags

[<a class="itemLink product-item" href="cerberus.html"><img alt="Cerberus Hemisphere Enhanced thumbnail" class="thumb" src="images/39d3266553462198bd2fbc4d18fbed17_cerberus_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="cerberus.html">
 <h3>Cerberus Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="schiaparelli.html"><img alt="Schiaparelli Hemisphere Enhanced thumbnail" class="thumb" src="images/08eac6e22c07fb1fe72223a79252de20_schiaparelli_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="schiaparelli.html">
 <h3>Schiaparelli Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="syrtis.html"><img alt="Syrtis Major Hemisphere Enhanced thumbnail" class="thumb" src="images/55a0a1e2796313fdeafb17c35925e8ac_syrtis_major_enhanced.tif_thumb.png"/></a>,
 <a class="itemLink product-item" href="syrtis.html">
 <h3>Syrtis Major Hemisphere Enhanced</h3>
 </a>,
 <a class="itemLink product-item" href="valles.html"><

In [61]:
#3. Write code to retrieve the titles for each hemisphere (and then image urls)
image_urls = []

for i in a_tags:
    hemisphere = {}
    # Get Hemisphere title
    hemisphere['title'] = browser.find_by_tag('h3')
    
    #Get image urls
    #links = browser.links.find_by_text('itemLink product-item')[i].click()
#    links = browser.links.find_by_partial_text('.html')

    #links = browser.find_by_css('.product-item img')[i].click()


ElementDoesNotExist: no elements could be found with link by partial text ".html"

In [36]:
# We have to find the elements on each loop to avoid a stale element exception
    browser.find_by_text('a.product-item img')[i].click()
    
    # Next, we find the Sample image anchor tag and extract the href
    sample_elem = browser.links.find_by_text('Sample').first
    hemisphere['img_url'] = sample_elem['href']

IndentationError: unexpected indent (<ipython-input-36-54e8ce894139>, line 2)

In [ ]:
a_tags = html_soup.find('div', class_='item')
a_tags

In [ ]:
# Scrape the Title
title = html_soup.find('h3').text
title

In [ ]:
#browser.visit("https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars")

In [ ]:
# html = browser.html
# hemispheres_html = soup(html, 'html.parser')
# hemispheres_list = []
# hemispheres_atags= hemispheres_html.find_all("a", class_= "itemLink")
# for i in range(0, len(hemispheres_atags)):
#     if i % 2 != 0:
#         browser.visit(f'https://astrogeology.usgs.gov{hemispheres_atags[i]["href"]}')
#         html = browser.html
#         hemisphere_html = soup(html, 'html.parser')
#         hemisphere_dict = {
#             "title": hemispheres_atags[i].find("h3").text,
#             "img_url": "https://upload.wikimedia.org/wikipedia/commons/0/02/OSIRIS_Mars_true_color.jpg"
#         }
#         hemispheres_list.append(hemisphere_dict)
# hemisphere_dict


In [ ]:
# end the automated browsing session
#browser.quit()